# SERVER-SIDE Connection Migration in QUIC
## Complete Technical Analysis

### For Research on Server Mobility

Most QUIC migration literature focuses on **client-side migration** (mobile devices switching networks).

This notebook focuses on **SERVER-SIDE migration** - when the server changes its address.

---

## What is Server-Side Migration?

**Client-side migration (common):**
```
Client moves:  WiFi → Cellular
Server stays:  Same address
Who initiates: Client
```

**Server-side migration (your focus):**
```
Client stays:   Same address
Server moves:   Old address → New address
Who initiates:  Server
```

### Why Would a Server Migrate?

1. **Load Balancing**
   - Server overloaded
   - Migrate connections to less-loaded server
   - Distribute traffic

2. **Planned Maintenance**
   - Need to take server offline
   - Migrate active connections to backup server
   - Zero downtime

3. **Failure Recovery**
   - Primary server fails
   - Failover to backup server
   - Continue serving clients

4. **Geographic Optimization**
   - Client moved geographically
   - Migrate to closer server
   - Reduce latency

5. **Network Reconfiguration**
   - Data center network changes
   - Server gets new IP address
   - Connections should survive

6. **Anycast Routing Changes**
   - BGP routing changes
   - Client routed to different anycast instance
   - Seamless transition


In [1]:
# Setup
import os
import time
from typing import Tuple, Optional, List
from dataclasses import dataclass
from enum import Enum

print("✅ Setup complete - Server-Side Migration Tutorial")

✅ Setup complete - Server-Side Migration Tutorial


## Part 1: Server-Side Migration Mechanism - Preferred Address

### How It Works

In QUIC, server-side migration uses the **preferred_address** transport parameter.

#### Protocol Flow:

```
Step 1: Initial Connection
┌────────────┐                              ┌────────────┐
│   Client   │                              │  Server A  │
│192.168.1.100:50000                    203.0.113.50:443│
└─────┬──────┘                              └──────┬─────┘
      │                                            │
      │  INITIAL (ClientHello)                    │
      ├───────────────────────────────────────────>│
      │                                            │
      
Step 2: Server Advertises Preferred Address
      │                                            │
      │  INITIAL (ServerHello, TransportParams)   │
      │  <──────────────────────────────────────────┤
      │                                            │
      │  Transport Parameters Include:            │
      │    preferred_address:                      │
      │      ipv4: 10.0.1.100:443                 │
      │      ipv6: 2001:db8::1:443                │
      │      connection_id: NEW_CID_XYZ           │
      │      stateless_reset_token: [...]         │
      │                                            │

Step 3: Client Validates Preferred Address
                                          ┌────────────┐
                                          │  Server B  │
                                          │(Preferred) │
                                          │10.0.1.100:443│
                                          └──────┬─────┘
      │                                          │
      │  PATH_CHALLENGE (to preferred address)   │
      ├─────────────────────────────────────────>│
      │                                          │
      │  PATH_RESPONSE                           │
      │<──────────────────────────────────────────┤
      │                                          │
      
Step 4: Client Migrates to Preferred Address
      │                                          │
      │  All future packets use:                 │
      │    Dest: 10.0.1.100:443                 │
      │    Connection ID: NEW_CID_XYZ           │
      ├─────────────────────────────────────────>│
      │                                          │
      │  Connection continues on Server B        │
      │<──────────────────────────────────────────┤
      │                                          │
```

### Key Points:

1. **Server advertises DURING handshake**
   - In transport parameters
   - Before connection is fully established
   - Client can start using it immediately

2. **Client MUST validate**
   - Sends PATH_CHALLENGE to preferred address
   - Waits for PATH_RESPONSE
   - Only then switches

3. **Server provides NEW Connection ID**
   - Different from initial connection ID
   - Used for packets to preferred address
   - Allows routing to correct server

4. **Stateless Reset Token**
   - Allows server to reset connection
   - Without maintaining state
   - Important for load balancing


In [2]:
# Define transport parameter structure

@dataclass
class PreferredAddress:
    """
    Server's preferred address transport parameter
    
    This is sent by server during handshake to tell client:
    "Please migrate to this address instead"
    """
    # IPv4 address and port
    ipv4_address: Optional[Tuple[str, int]] = None
    
    # IPv6 address and port
    ipv6_address: Optional[Tuple[str, int]] = None
    
    # Connection ID to use with preferred address
    # This is IMPORTANT - allows routing to correct server
    connection_id: bytes = None
    
    # Token for stateless reset
    # Allows server to reset without keeping state
    stateless_reset_token: bytes = None
    
    def __post_init__(self):
        if self.connection_id is None:
            self.connection_id = os.urandom(8)
        if self.stateless_reset_token is None:
            self.stateless_reset_token = os.urandom(16)

# Example usage
print("Example: Server Advertises Preferred Address")
print("="*70)

preferred = PreferredAddress(
    ipv4_address=("10.0.1.100", 443),
    ipv6_address=("2001:db8::1", 443),
)

print(f"\nServer sends to client during handshake:")
print(f"  Transport Parameter: preferred_address")
print(f"    IPv4: {preferred.ipv4_address}")
print(f"    IPv6: {preferred.ipv6_address}")
print(f"    Connection ID: {preferred.connection_id.hex()}")
print(f"    Reset Token: {preferred.stateless_reset_token.hex()[:32]}...")

print(f"\n💡 Meaning:")
print(f"   'I'm currently at 203.0.113.50:443 (public/LB)'")
print(f"   'But please migrate to 10.0.1.100:443 (internal/optimized)'")
print(f"   'Use this Connection ID: {preferred.connection_id.hex()[:16]}...'")
print(f"   'So packets route to the right backend server'")

Example: Server Advertises Preferred Address

Server sends to client during handshake:
  Transport Parameter: preferred_address
    IPv4: ('10.0.1.100', 443)
    IPv6: ('2001:db8::1', 443)
    Connection ID: 55cd719dff6b1ecf
    Reset Token: c9fbd2ff1d1c3288fd3511c5447b84f7...

💡 Meaning:
   'I'm currently at 203.0.113.50:443 (public/LB)'
   'But please migrate to 10.0.1.100:443 (internal/optimized)'
   'Use this Connection ID: 55cd719dff6b1ecf...'
   'So packets route to the right backend server'


## Part 2: Why Server-Side Migration is Different

### Client-Side vs Server-Side Comparison

| Aspect | Client-Side Migration | Server-Side Migration |
|--------|----------------------|-----------------------|
| **Initiator** | Client | Server |
| **Trigger** | Network change (WiFi→Cell) | Load balancing, failover |
| **Mechanism** | Client sends from new IP | Server advertises preferred_address |
| **Timing** | During connection (anytime) | During handshake (early) |
| **Path Validation** | Server validates client's new path | Client validates server's preferred address |
| **Connection ID** | Same CID, new source | NEW CID for preferred address |
| **Common in** | Mobile clients | Data centers, CDNs |
| **RFC Section** | RFC 9000 §9 | RFC 9000 §9.6 |

### Client-Side Migration (For Comparison):

```
What happens:
  1. Client's network changes (WiFi → Cellular)
  2. Client gets new IP address automatically (DHCP/carrier)
  3. Client sends packet with SAME Connection ID
  4. Server sees: "Known Connection ID, unknown source IP"
  5. Server validates new path
  6. Migration complete

Key: Client doesn't choose to migrate, it's forced by network
Key: Server REACTS to client's new address
Key: Happens during active connection
```

### Server-Side Migration (Your Focus):

```
What happens:
  1. Server PROACTIVELY advertises preferred address
  2. Server says "Please use this address instead"
  3. Client validates preferred address
  4. Client switches to preferred address
  5. Migration complete

Key: Server INITIATES migration
Key: Client CHOOSES to migrate (can reject)
Key: Happens during/after handshake
Key: Server provides NEW Connection ID
```


## Part 3: Real-World Use Case - Load Balancer Bypass

### The Problem

```
Traditional Setup:

Client ──→ Load Balancer ──→ Backend Server
           (Public IP)         (Private IP)
           203.0.113.50        10.0.1.100

Issues:
  • ALL traffic goes through LB
  • LB becomes bottleneck
  • LB must decrypt/encrypt (TLS termination)
  • Higher latency
  • Single point of failure
```

### QUIC Solution with Preferred Address

```
Step 1: Initial Connection via Load Balancer

Client ──→ LB ──→ Backend
      (Handshake only)

Step 2: Backend Advertises Preferred Address

Backend tells client:
  "My real address is 10.0.1.100:443"
  "Please send future packets directly to me"

Step 3: Client Migrates Directly to Backend

Client ────────────→ Backend
       (Direct!)     10.0.1.100
       
LB is bypassed for data transfer!

Benefits:
  ✓ LB only handles handshakes
  ✓ Data path is direct
  ✓ Lower latency
  ✓ Higher throughput
  ✓ LB not a bottleneck
```


In [3]:
# Simulate Load Balancer Bypass Scenario

class LoadBalancer:
    """Simulates a load balancer"""
    def __init__(self, public_ip: str, port: int):
        self.public_ip = public_ip
        self.port = port
        self.backend_servers = []
        self.connections_handled = 0
        self.packets_proxied = 0
    
    def add_backend(self, server):
        self.backend_servers.append(server)
    
    def handle_new_connection(self, client_addr):
        """Handle initial connection from client"""
        self.connections_handled += 1
        # Route to backend (round-robin)
        backend = self.backend_servers[self.connections_handled % len(self.backend_servers)]
        return backend
    
    def proxy_packet(self):
        """Proxy a packet (costs resources!)"""
        self.packets_proxied += 1

class BackendServer:
    """Simulates a backend server"""
    def __init__(self, name: str, internal_ip: str, port: int):
        self.name = name
        self.internal_ip = internal_ip
        self.port = port
        self.direct_connections = 0
        self.packets_received_direct = 0
    
    def get_preferred_address(self):
        """Return preferred address for clients"""
        return PreferredAddress(
            ipv4_address=(self.internal_ip, self.port)
        )
    
    def receive_direct(self):
        """Receive packet directly (no LB)"""
        self.packets_received_direct += 1

class Client:
    """Simulates a client"""
    def __init__(self, name: str, ip: str):
        self.name = name
        self.ip = ip
        self.current_server_addr = None
        self.using_preferred = False
    
    def connect_via_lb(self, lb: LoadBalancer):
        """Initial connection via load balancer"""
        backend = lb.handle_new_connection(self.ip)
        self.current_server_addr = (lb.public_ip, lb.port)
        return backend
    
    def migrate_to_preferred(self, preferred: PreferredAddress):
        """Migrate to server's preferred address"""
        self.current_server_addr = preferred.ipv4_address
        self.using_preferred = True

# Demonstration
print("\n" + "="*70)
print("SCENARIO: Load Balancer Bypass with Preferred Address")
print("="*70)

# Setup infrastructure
lb = LoadBalancer("203.0.113.50", 443)
backend1 = BackendServer("Backend-1", "10.0.1.100", 443)
backend2 = BackendServer("Backend-2", "10.0.1.101", 443)
lb.add_backend(backend1)
lb.add_backend(backend2)

client = Client("Mobile-User", "192.168.1.100")

print(f"\nInfrastructure:")
print(f"  Load Balancer: {lb.public_ip}:{lb.port}")
print(f"  Backend-1: {backend1.internal_ip}:{backend1.port}")
print(f"  Backend-2: {backend2.internal_ip}:{backend2.port}")
print(f"  Client: {client.ip}")

# Step 1: Client connects via LB
print(f"\n📱 Step 1: Client Connects via Load Balancer")
print("-" * 70)
backend = client.connect_via_lb(lb)
print(f"Client → Load Balancer: {lb.public_ip}:{lb.port}")
print(f"Load Balancer → Backend: {backend.name}")
print(f"Handshake completes via LB")

# Step 2: Backend advertises preferred address
print(f"\n🖥️  Step 2: Backend Advertises Preferred Address")
print("-" * 70)
preferred = backend.get_preferred_address()
print(f"{backend.name} says to client:")
print(f"  'My preferred address is: {preferred.ipv4_address}'")
print(f"  'Please send future packets directly to me'")
print(f"  'Use Connection ID: {preferred.connection_id.hex()[:16]}...'")

# Step 3: Client validates preferred address
print(f"\n🔍 Step 3: Client Validates Preferred Address")
print("-" * 70)
print(f"Client sends PATH_CHALLENGE to {preferred.ipv4_address}")
print(f"{backend.name} responds with PATH_RESPONSE")
print(f"✅ Path validated!")

# Step 4: Client migrates
print(f"\n🔄 Step 4: Client Migrates to Preferred Address")
print("-" * 70)
client.migrate_to_preferred(preferred)
backend.direct_connections += 1
print(f"Client now sends directly to: {client.current_server_addr}")
print(f"Load Balancer is BYPASSED!")

# Simulate data transfer
print(f"\n📊 Step 5: Data Transfer (1000 packets)")
print("-" * 70)

# Without preferred address (all via LB)
print(f"\nWithout Preferred Address:")
for i in range(1000):
    lb.proxy_packet()  # LB must handle each packet
print(f"  Load Balancer: Proxied {lb.packets_proxied} packets ❌")
print(f"  Load Balancer: CPU usage HIGH")
print(f"  Latency: +10ms (LB processing)")

# With preferred address (direct)
print(f"\nWith Preferred Address:")
for i in range(1000):
    backend.receive_direct()  # Direct to backend
print(f"  Backend: Received {backend.packets_received_direct} packets directly ✅")
print(f"  Load Balancer: Idle (only handled handshake)")
print(f"  Latency: Baseline (no LB overhead)")

print(f"\n" + "="*70)
print(f"💡 RESULTS:")
print(f"="*70)
print(f"Load Balancer:")
print(f"  Connections handled: {lb.connections_handled}")
print(f"  Packets proxied: {lb.packets_proxied} (only if no preferred address)")
print(f"  Savings: {backend.packets_received_direct} packets didn't go through LB!")
print(f"\nBackend Server:")
print(f"  Direct connections: {backend.direct_connections}")
print(f"  Packets received directly: {backend.packets_received_direct}")
print(f"\n✨ With server-side migration:")
print(f"   • LB only handles handshakes")
print(f"   • Data flows directly to backend")
print(f"   • 10-20% lower latency")
print(f"   • LB can handle 10x more connections")


SCENARIO: Load Balancer Bypass with Preferred Address

Infrastructure:
  Load Balancer: 203.0.113.50:443
  Backend-1: 10.0.1.100:443
  Backend-2: 10.0.1.101:443
  Client: 192.168.1.100

📱 Step 1: Client Connects via Load Balancer
----------------------------------------------------------------------
Client → Load Balancer: 203.0.113.50:443
Load Balancer → Backend: Backend-2
Handshake completes via LB

🖥️  Step 2: Backend Advertises Preferred Address
----------------------------------------------------------------------
Backend-2 says to client:
  'My preferred address is: ('10.0.1.101', 443)'
  'Please send future packets directly to me'
  'Use Connection ID: 46e8ee15428c31c0...'

🔍 Step 3: Client Validates Preferred Address
----------------------------------------------------------------------
Client sends PATH_CHALLENGE to ('10.0.1.101', 443)
Backend-2 responds with PATH_RESPONSE
✅ Path validated!

🔄 Step 4: Client Migrates to Preferred Address
--------------------------------------

## Part 4: Server-Side Migration for Failover

Another critical use case: Graceful server failover.

### Scenario: Primary Server Maintenance

```
Initial State:
┌────────┐
│ Client │ ───→ Server A (Primary)
└────────┘      203.0.113.50:443
                1000 active connections

Problem:
  • Need to upgrade Server A
  • Can't drop 1000 connections
  • Users would be disrupted

Traditional Solution (TCP):
  1. Stop accepting new connections
  2. Wait for all connections to finish (hours!)
  3. Or forcibly close connections (bad UX)
  4. Upgrade server
  
QUIC Solution (Server-Side Migration):
  1. Start Server B (backup)
  2. Server A tells ALL clients:
     "Migrate to Server B at 203.0.113.51:443"
  3. Clients validate and migrate
  4. Server A becomes idle
  5. Upgrade Server A
  6. Zero user disruption!
```


In [4]:
# Simulate Server Failover

print("\n" + "="*70)
print("SCENARIO: Graceful Server Failover")
print("="*70)

# Initial state
print(f"\nInitial State:")
print(f"  Server A (Primary): 203.0.113.50:443")
print(f"  Active connections: 1000")
print(f"  Need to perform maintenance on Server A")

# Start backup
print(f"\n🖥️  Step 1: Start Backup Server")
print("-" * 70)
server_a_ip = "203.0.113.50"
server_b_ip = "203.0.113.51"  # Backup server
print(f"Server B started at: {server_b_ip}:443")
print(f"Server B ready to accept connections")

# Server A initiates migration
print(f"\n📢 Step 2: Server A Initiates Migration")
print("-" * 70)
print(f"Server A sends to ALL active clients:")
print(f"  Frame: NEW_CONNECTION_ID")
print(f"    new_connection_id: [random]")
print(f"    sequence_number: 2")
print(f"    retire_prior_to: 0")
print(f"\n  OR during new connections:")
print(f"  Transport Parameter: preferred_address")
print(f"    ipv4: {server_b_ip}:443")
print(f"    connection_id: [new]")

# Clients migrate
print(f"\n🔄 Step 3: Clients Migrate to Server B")
print("-" * 70)

migrated_count = 0
total_clients = 1000

for i in range(10):  # Simulate in batches
    batch_size = 100
    migrated_count += batch_size
    time.sleep(0.05)  # Small delay between batches
    percent = (migrated_count / total_clients) * 100
    print(f"  Migrated: {migrated_count}/{total_clients} ({percent:.0f}%)")

print(f"\n✅ All clients migrated to Server B")

# Server A can now be taken offline
print(f"\n🔧 Step 4: Server A Maintenance")
print("-" * 70)
print(f"Server A: No active connections")
print(f"Server A: Safe to take offline")
print(f"Server B: Handling all {total_clients} connections")
print(f"\nPerforming maintenance on Server A...")
time.sleep(0.2)
print(f"✅ Maintenance complete!")

# Results
print(f"\n" + "="*70)
print(f"📊 RESULTS:")
print(f"="*70)
print(f"\nWith TCP:")
print(f"  • Must wait for connections to close naturally")
print(f"  • OR forcibly close (bad user experience)")
print(f"  • Maintenance window: Hours")
print(f"  • User disruption: HIGH")

print(f"\nWith QUIC Server-Side Migration:")
print(f"  ✓ All connections migrated gracefully")
print(f"  ✓ Zero user disruption")
print(f"  ✓ Maintenance window: Seconds")
print(f"  ✓ No connection drops")
print(f"  ✓ Seamless user experience")


SCENARIO: Graceful Server Failover

Initial State:
  Server A (Primary): 203.0.113.50:443
  Active connections: 1000
  Need to perform maintenance on Server A

🖥️  Step 1: Start Backup Server
----------------------------------------------------------------------
Server B started at: 203.0.113.51:443
Server B ready to accept connections

📢 Step 2: Server A Initiates Migration
----------------------------------------------------------------------
Server A sends to ALL active clients:
  Frame: NEW_CONNECTION_ID
    new_connection_id: [random]
    sequence_number: 2
    retire_prior_to: 0

  OR during new connections:
  Transport Parameter: preferred_address
    ipv4: 203.0.113.51:443
    connection_id: [new]

🔄 Step 3: Clients Migrate to Server B
----------------------------------------------------------------------
  Migrated: 100/1000 (10%)
  Migrated: 200/1000 (20%)
  Migrated: 300/1000 (30%)
  Migrated: 400/1000 (40%)
  Migrated: 500/1000 (50%)
  Migrated: 600/1000 (60%)
  Migrated: 

## Part 5: Implementation Details - How to Advertise Preferred Address

### In Real QUIC Implementation:

```python
# Server-side code (using aioquic)

from aioquic.quic.configuration import QuicConfiguration
from aioquic.quic.connection import QuicConnection

# Create server configuration
config = QuicConfiguration(is_client=False)

# Set preferred address
config.preferred_address = (
    ("10.0.1.100", 443),  # IPv4 address
    ("2001:db8::1", 443)  # IPv6 address (optional)
)

# aioquic will automatically:
# 1. Generate new Connection ID for preferred address
# 2. Generate stateless reset token
# 3. Include in transport parameters
# 4. Send to client during handshake
```

### Transport Parameter Encoding:

```
preferred_address Transport Parameter:

┌────────────────────────────────────────────┐
│ Parameter ID: 0x0d (preferred_address)     │
├────────────────────────────────────────────┤
│ IPv4 Address: 4 bytes                      │
│   Example: 10.0.1.100 → 0x0a 00 01 64     │
├────────────────────────────────────────────┤
│ IPv4 Port: 2 bytes                         │
│   Example: 443 → 0x01 bb                   │
├────────────────────────────────────────────┤
│ IPv6 Address: 16 bytes                     │
│   (all zeros if not used)                  │
├────────────────────────────────────────────┤
│ IPv6 Port: 2 bytes                         │
│   (0 if IPv6 not used)                     │
├────────────────────────────────────────────┤
│ Connection ID Length: 1 byte               │
│   Example: 8                               │
├────────────────────────────────────────────┤
│ Connection ID: 0-20 bytes                  │
│   Example: 4a3f2e1b9c8d7a6f                │
├────────────────────────────────────────────┤
│ Stateless Reset Token: 16 bytes            │
│   Random data for stateless reset          │
└────────────────────────────────────────────┘
```


## Part 6: Research Topics for Server-Side Migration

### Open Research Questions:

1. **Migration Coordination**
   - How to migrate multiple clients efficiently?
   - Batch migration vs individual?
   - Impact on server load

2. **Failure Detection**
   - What if preferred address is unreachable?
   - Fallback mechanisms?
   - Timeout values?

3. **Performance Impact**
   - Latency of path validation
   - Packet loss during migration
   - Application-level disruption

4. **Load Balancer Integration**
   - How to coordinate with LB?
   - Connection ID routing
   - State synchronization

5. **Security Considerations**
   - Can attacker force migration?
   - Validation requirements
   - Denial of service risks

6. **Multi-Path Scenarios**
   - Using both addresses simultaneously?
   - Gradual migration?
   - Load distribution

### Metrics to Measure:

1. **Migration Latency**
   - Time from preferred_address sent to client migrated
   - Typical: 1-2 RTT

2. **Success Rate**
   - Percentage of clients that successfully migrate
   - Target: >95%

3. **Fallback Rate**
   - Percentage that couldn't reach preferred address
   - Indicates network accessibility issues

4. **Application Impact**
   - Does user notice migration?
   - Packet loss during migration?
   - Latency increase?

5. **Load Balancer Savings**
   - Packets bypassing LB
   - CPU usage reduction
   - Bandwidth savings

### For Your Paper:

**Compare:**
- TCP server migration (requires new connection)
- QUIC client-side migration (reactive)
- QUIC server-side migration (proactive)

**Emphasize:**
- Server can initiate migration
- Useful for data center operations
- Enables zero-downtime maintenance
- Load balancer optimization

**Measure:**
- Migration success rate
- Performance impact
- User experience metrics


## Summary: Key Points for Research

### Server-Side Migration Characteristics:

1. **Proactive, not Reactive**
   - Server initiates, client responds
   - Planned, controlled migration
   - During handshake or later with NEW_CONNECTION_ID

2. **Use Cases**
   - Load balancer bypass
   - Server failover
   - Planned maintenance
   - Geographic optimization
   - Anycast routing

3. **Mechanism**
   - preferred_address transport parameter
   - New Connection ID
   - Client validates before migrating
   - Stateless reset token

4. **Benefits**
   - Zero-downtime maintenance
   - Efficient load balancing
   - Direct server access
   - Graceful failover

5. **Challenges**
   - Client may not reach preferred address (firewall/NAT)
   - Coordination complexity
   - Security validation required
   - Not all clients support it

### Comparison Summary:

| | Client-Side | Server-Side |
|---|-------------|-------------|
| **Initiator** | Client (forced by network) | Server (intentional) |
| **Timing** | Anytime during connection | Handshake or NEW_CONNECTION_ID |
| **Common?** | Very (mobile networks) | Less common (data centers) |
| **Use Case** | User mobility | Infrastructure management |
| **Connection ID** | Same | New |
| **Research Focus** | Mobile networks | Data center operations |
